## <CENTER>  Anpassningar i ROOT: tips och exempel

In [ ]:
from ROOT import TH1F,TCanvas,TF1,TLegend,kRed,kBlue,kGreen,kBlack

In [ ]:
# Gör ett histogram 
histo = TH1F("histo", "; x; Number of events", 20, -3., 3.)
# antal entries
N = 500
# fyll histogrammet med slumpade tal från en funktion
fname="expo" # ta t.ex. "expo", en fördefinierad funktion med två parametrar = e^([parameter0] + [parameter1]*x). Andra exempel: "gaus", "landau"...
histo.FillRandom(fname, N) # fyll N gånger

In [ ]:
# Förbered för att rita histogrammet
%jsroot on

In [ ]:
canvas = TCanvas("canvas", "", 800, 600)

In [ ]:
# Rita histogrammet
histo.Draw("e1p")
canvas.Draw()

### Gör en anpassning 
Nu ska vi göra en anpassningen till data i histogrammet ovan. Syntax är `TH1F::Fit(fcn, "fit_opt", "draw_opt", xlow, xup)`
där 
* `fcn` är antingen en sträng med namnet på en funktion (ROOT::TF1-objekt), eller själva funktionsobjektet.
* sedan kommer anpassnings- och grafik-alternativ
* sist är range i vilken anpassning ska göras

Se vidare dokumentation här https://root.cern.ch/doc/master/classTH1.html#a7e7d34c91d5ebab4fc9bba3ca47dabdd. Framförallt kan man läsa om de olika alternativen för att kontrollera anpassningen (mer om detta snart).

Vi testar först att helt enkelt att anpassa till den fördefinierade funktionen "expo" som användes för att fylla histogrammet (borde per definition ge en bra anpassning).
Genom att ropa på Fit görs anpassningen och den resulterande kurvan ritas upp. 
Per default används en viktad minska-kvadrat-anpassning, även kallad $\chi^2$-anpassning (se "Statistik för fysikexperiment.pdf" på Canvas)

In [ ]:
# Gör anpassning med samma funktion som histogrammet fylldes 
histo.Fit(fname) # gör anpassningen. 
print("\n****** Fit results: ")
canvas.Draw() # titta på den resulterande kurvan


Ovan ser ni resultaten från anpassningen. Titta t.ex. på `STATUS` som visar om anpassningen konvergerat (hittat minimum) eller inte. Ni ser de resulterade best-fit-värdena på parametrarna och deras respektive osäkerheter.

Skriv ut den relativa osäkerheten på Slope-parametern nedan:

In [ ]:
# print("Relative uncertainty Slope = ..."
print("Relative uncertainty Slope = {:.3f}".format(abs(histo.GetFunction(fname).GetParError(0) / histo.GetFunction(fname).GetParameter(0) )))

### Testa nu att gå tillbaka och öka antalet entries i histogrammet till det dubbla. Kör om anpassningen. Vad händer med osäkerheten? Om ni ökar med en faktor fyra? Förklara.

___________________________________________________________________________

### Konfigurering av anpassning
Default är som sagt att anpassa genom minimering av den viktade minsta-kvadrat-summan. Detta bygger på antagandet att mätvärdena kommer från normalfördelningen. I vårt fall gäller att de är Poissonfördelade–men vi vet att Poissonfördelningen närmar sig normalfördelningen då väntevärdet är stort.

Man kan också ge fit option = "L" för att göra en maximum likelihood-anpassning. I denna maximeras istället en likelihood som är lika med en produkt av Poisson-termer där varje term representerar antal counts i en bin.

Låt oss jämföra de två alternativen med ett exempel. Betrakta en modell som är en rät linje. Vi genererar ett antal event och ritar upp ett histogram, som motsvarar observationen av ett experiment. Målet är att mäta linjens lutning genom att göra en anpassning.

In [ ]:
# modell
fmodel = TF1("lin", "[0]*x+[1]", 3., 10.)
# sätt parametervärden
fmodel.SetParameters(1., 1.)
# skapa ett histogram
hist = TH1F("exp", "; x; Number of events", 10, 3., 10.)
n=30
hist.FillRandom("lin", n) # fyll n gånger med slumpade värden från linjen

# anpassa med default minsta-kvadrat-metod
hist.Fit("lin", "", "e1")
# spara funktionen och rita upp den i blått (så att den inte försvinner när vi anpassar samma funktion igen senare)
fresultLeastSquares=hist.GetFunction("lin").Clone("linLeastSquares") # "Clone" skapar en kopia av funktionen
fresultLeastSquares.SetLineColor(kBlue)
fresultLeastSquares.Draw("SAME")
print("\nSlope least squares = {:.3f}".format(fresultLeastSquares.GetParameter(0)))

# anpassa igen men denna gång med option "L". Den resulterande kurvan kommer att visas i rött
hist.Fit("lin", "L", "e1 SAME")
print("Slope likelihood = {:.3f}\n".format(hist.GetFunction("lin").GetParameter(0)))
canvas.Draw()

### Som ni ser får vi inte samma lutning med de olika alternativen för anpassning. Testa vad som händer om ni ökar antalet event i histogrammet. Förklara.

_________________


### Definiera egna funktioner

In [ ]:
# OBS interaktivitet funkar tyvärr inte för alla funktioner. Stäng av för att vara säker på att allt ritas som det ska.
%jsroot off

In [ ]:
# definiera en egen funktion, utgå från a*(x-b)^2
### Alternativ 1: skriv formeln direkt med "TFormula"-syntax
f1 = TF1("f1", "[0]*(x-[1])*(x-[1])", -10., 10.) # [0] är första parametern, [1] är andra osv.
                                                 # de två sista argumenten är range för funktionen
### Alternativ 2: skriv m.h.a. redan existerande funktioner (fördefinierade eller egendefinierade)
f2 = TF1("f2", "gaus + f1", -10., 10.)
### Alternativ 3: med hjälp av egen-definierad python-funktion
def myfunc(x, params):
    x=x[0]
    a=params[0]; b=params[1]
    return a*(x-b)**2
f3 = TF1("f3", myfunc, -10., 10., 2) # det sista argumentet anger antal parametrar
### sätt parameter-värden
f1.SetParameters(1., 2.)
f2.SetParameters(100., -2., 4., 1., 2.) # första tre parametrarna är normalfördelningen, de sista f1-parametrarna
f3.SetParameters(1., 3.)
### sätt tvingande gränser för parametrarna (användbart vid anpassningar)
f1.SetParLimits(0, -3., 5.) # parameter0 måste ligga inom [-3., 5.] 
### sätt grafik
f1.SetLineColor(kBlue)
f2.SetLineColor(kBlack)
f3.SetLineColor(kRed)
### rita
f1.Draw()
f2.Draw("SAME")
f3.Draw("SAME")
canvas.Draw()

___________________________________________________

### Rita en legend
Generellt ska figurer ha en "legend" som visar vad de olika uppritade kurvorna/datapunkterna representerar. Nedan följer ett exempel. Se t.ex. https://root.cern.ch/doc/master/classTLegend.html för ytterligare dokumentation 

In [ ]:
# skapa legend-objektet
leg = TLegend(0.5, 0.5, 0.8, 0.8) # argumenten är x1,y1,x2,y2-koordinater, givna som andel av canvas
# lägg till kurvorna vi ritade ovan. Syntax är TLegend::AddEntry(drawn_object, title, plot_style)
leg.AddEntry(f1, "Parabola, defined with TFormula", "l") # "l" står för "line", alltså kurva
leg.AddEntry(f2, "Parabola + Gaussian", "l")
leg.AddEntry(f3, "Parabola, user defined", "l") # hade vi ritat ett histogram kan vi istället använda "p" 
# gör legend snyggare
# ...
# rita legenden
leg.Draw()
canvas.Draw()



Se till att texten i legenden inte är för liten. Kan styras med `leg.SetTextSize(x)` där `x` är andel av canvas-storleken. Testa att förstora texten. Kanten kan tas bort med `leg.SetLegendBorderSize(0)`.

______________


### Besvara följande frågor. Ni kommer att behöva dem för att i rapporten förklara hur anpassningen görs. 
* För histogram ser ni osäkerheter på varje datapunkt. ROOT har automatiskt räknat ut dem till... vadå?
* Vad bestämmer osäkerheten på de anpassade parametrarna? Hur kan de minskas?
* ROOT anpassar per default med viktad minsta-kvadrat-metoden ($\chi^2$-anpassning). När är detta korrekt? Vad händer om vi anpassar med option "L"?
